In [1]:
import pandas as pd 
import numpy as np

In [3]:
out = pd.read_csv('Documents/titanic/gender_submission.csv')
out.tail(10)

,PassengerId,Survived
408,1300,1
409,1301,1
410,1302,1
411,1303,1
412,1304,1
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0


In [2]:
df = pd.read_csv('Documents/titanic/train.csv')
test = pd.read_csv('Documents/titanic/test.csv')


In [3]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
for c in df.Sex.unique().tolist():
    tab = df[df.Sex == c].Survived.value_counts()
    print(f'{c}\n{tab}\nChance of survive: {tab[1]/(tab[1]+tab[0])}\n')

male
0    468
1    109
Name: Survived, dtype: int64
Chance of survive: 0.18890814558058924

female
1    233
0     81
Name: Survived, dtype: int64
Chance of survive: 0.7420382165605095



In [5]:
first_quantile_age = df[df.Age < 20.125]
fqa_surv = first_quantile_age.Survived.value_counts()
print(f'&lt I Quantile:\n{fqa_surv}\nChance of survive: {fqa_surv[1]/(fqa_surv[1]+fqa_surv[0]):.5f}\n')

third_quantile_age = df[df.Age > 38]
tqa_surv = third_quantile_age.Survived.value_counts()
print(f'&gt III Quantile:\n{tqa_surv}\nChance of survive: {tqa_surv[1]/(tqa_surv[1]+tqa_surv[0]):.5f}\n')



&lt I Quantile:
0    97
1    82
Name: Survived, dtype: int64
Chance of survive: 0.45810

&gt III Quantile:
0    111
1     66
Name: Survived, dtype: int64
Chance of survive: 0.37288



In [56]:
s = df[df.Age <= 28].Survived.value_counts()
s[1]/(s[1]+s[0])

0.40883977900552487

In [52]:
s = df[(28 < df.Age) & df.Age < 38].Survived.value_counts()
s[1]/(s[1]+s[0])

0.38383838383838381

In [54]:
df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [6]:
df = df.drop(['Ticket', 'Cabin'], axis=1)
test = test.drop(['Ticket', 'Cabin'], axis=1)
full_dataset = [df, test]


In [7]:
for dataset in full_dataset:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df['Title'], df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [8]:
for dataset in full_dataset:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [9]:
df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [11]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in full_dataset:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [12]:
df = df.drop(['Name', 'PassengerId'], axis=1)
test = test.drop(['Name'], axis=1)
full_dataset = [df, test]


In [13]:
for dataset in full_dataset:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [18]:
guess_ages = np.zeros((2,3))

for dataset in full_dataset:
    for i in range(2):
        for j in range(3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(2):
        for j in range(3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

In [19]:
df['AgeBand'] = pd.cut(df['Age'], 5)
df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.337374
2,"(32.0, 48.0]",0.412037
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [20]:
for dataset in full_dataset:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']

In [21]:
df = df.drop(['AgeBand'], axis=1)
full_dataset = [df, test]

In [22]:
for dataset in full_dataset:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [23]:
for dataset in full_dataset:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [24]:
df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [25]:
df = df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test= test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
full_dataset = [df, test]

In [26]:
for dataset in full_dataset:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

In [27]:
freq_port = df.Embarked.dropna().mode()[0]

for dataset in full_dataset:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [28]:
for dataset in full_dataset:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [29]:
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)

In [30]:
df['FareBand'] = pd.qcut(df['Fare'], 4)
df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [31]:
for dataset in full_dataset:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

df = df.drop(['FareBand'], axis=1)
full_dataset = [df, test]

In [33]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=0.2)
X_train, X_val = df_train.drop('Survived', axis=1), df_val.drop('Survived', axis=1)
y_train, y_val = df_train['Survived'], df_val['Survived']

In [35]:
X_val

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
441,3,0,1,1,0,1,1,3
104,3,0,2,1,0,1,0,6
182,3,0,0,3,0,4,0,0
727,3,1,1,0,2,2,1,3
765,1,1,3,3,0,3,0,3
579,3,0,1,1,0,1,1,3
246,3,1,1,0,0,2,1,3
193,2,0,0,2,0,4,0,0
176,3,0,1,2,0,4,0,3
696,3,0,2,1,0,1,1,6


In [56]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,16 mins 31 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,1 month and 9 days
H2O cluster name:,H2O_from_python_dawg_nluw9b
H2O cluster total nodes:,1
H2O cluster free memory:,1.149 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [42]:
training_cols = X_train.columns.tolist()
response = y_train.name

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class,Survived
854,2,1,2,2,0,3,0,4,0
575,3,0,1,2,0,1,1,3,0
636,3,0,1,1,0,1,1,3,0
414,3,0,2,1,0,1,1,6,1
329,1,1,0,3,1,2,0,0,1
760,3,0,1,2,0,1,1,3,0
794,3,0,1,0,0,1,1,3,0
200,3,0,1,1,0,1,1,3,0
273,1,0,2,2,1,1,0,2,0
393,1,1,1,3,1,2,0,1,1


In [60]:
h20_df = h2o.H2OFrame(df)
h20_df['Survived'] = h20_df['Survived'].asfactor()
df_train, df_test = h20_df.split_frame(ratios=[0.8])

/home/dawg/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [61]:
# Define model
model = H2OGradientBoostingEstimator(ntrees=50, max_depth=20, nfolds=10)

# Train model
model.train(x=training_cols, y=response, training_frame=df_train)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [62]:
performance = model.model_performance(test_data=df_test)

performance


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.13377310566939024
RMSE: 0.3657500590148828
LogLoss: 0.43881593571649646
Mean Per-Class Error: 0.18571428571428572
AUC: 0.8455844155844157
Gini: 0.6911688311688313
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4479023147563183: 


,0,1,Error,Rate
0,99.0,11.0,0.1,(11.0/110.0)
1,19.0,51.0,0.2714,(19.0/70.0)
Total,118.0,62.0,0.1667,(30.0/180.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4479023,0.7727273,39.0
max f2,0.0569700,0.7876712,75.0
max f0point5,0.6881556,0.8547009,28.0
max accuracy,0.5265398,0.8388889,34.0
max precision,0.9959492,1.0,0.0
max recall,0.0530595,1.0,77.0
max specificity,0.9959492,1.0,0.0
max absolute_mcc,0.5265398,0.6617525,34.0
max min_per_class_accuracy,0.2856498,0.7428571,45.0
max mean_per_class_accuracy,0.4479023,0.8142857,39.0


Gains/Lift Table: Avg response rate: 38.89 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0111111,0.9922638,2.5714286,2.5714286,1.0,1.0,0.0285714,0.0285714,157.1428571,157.1428571
,2,0.0333333,0.9891094,2.5714286,2.5714286,1.0,1.0,0.0571429,0.0857143,157.1428571,157.1428571
,3,0.0333333,0.9890442,0.0,2.5714286,0.0,1.0,0.0,0.0857143,-100.0,157.1428571
,4,0.05,0.9889332,2.5714286,2.5714286,1.0,1.0,0.0428571,0.1285714,157.1428571,157.1428571
,5,0.05,0.9872327,0.0,2.5714286,0.0,1.0,0.0,0.1285714,-100.0,157.1428571
,6,0.1,0.9404486,2.2857143,2.4285714,0.8888889,0.9444444,0.1142857,0.2428571,128.5714286,142.8571429
,7,0.15,0.8844445,2.5714286,2.4761905,1.0,0.9629630,0.1285714,0.3714286,157.1428571,147.6190476
,8,0.2,0.7348660,2.5714286,2.5,1.0,0.9722222,0.1285714,0.5,157.1428571,150.0
,9,0.3111111,0.4902201,1.5428571,2.1581633,0.6,0.8392857,0.1714286,0.6714286,54.2857143,115.8163265
,10,0.4222222,0.3093650,0.5142857,1.7255639,0.2,0.6710526,0.0571429,0.7285714,-48.5714286,72.5563910


In [63]:
model.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.317424909582525: 


,0,1,Error,Rate
0,380.0,59.0,0.1344,(59.0/439.0)
1,41.0,231.0,0.1507,(41.0/272.0)
Total,421.0,290.0,0.1406,(100.0/711.0)
